In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import math 
import scipy
from scipy import stats
import sys
import time
from tqdm import tqdm
%matplotlib inline
np.seterr(divide = 'warn') 

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
Inital = 15 #Initial Price of the Assset
Strike  = 17
r       = 0.0255  #Risk Free interest rate
Sigmat = 0.35     #Inital In-volatility estimate 
Expiry  = 252       #Time Period
Steps   = 252

npaths  = 2000     # No of MC Simulations or Replications 
nparticle  = 100   # No of particles 
nsteps = 252       #

Rho     = -0.035 #Codependence between the shareprice and volatility process
Alpha   = 0.25  #Volatility Mean Revision Rate 
Beta    = np.log10(0.20) #Volatility Mean Revision Level
Gamma   = 2.10  #Volatility of Volatility
Lambda  = -1.0  #Market Price of Volatility or Volatility risk premium (also it is a constant )

Delta   = Expiry/Steps
BetaStar = Beta - ((Lambda*Gamma)/Alpha)
SqrtDelta = np.sqrt(Delta)
RhoC = np.sqrt(1-(Rho**2))
Expat = np.exp(-Alpha*Delta)
Exprt = np.exp(-r*Delta)
SqrtAlpha = np.sqrt(2*Alpha)
InvolSD = Gamma* np.sqrt(1- Expat*Expat)/SqrtAlpha

print('Delta:',Delta,'\nBetaStar:',BetaStar,'\nSqrtDelta:',SqrtDelta,'\nRhoC:',RhoC,
      '\nExpat:',Expat,'\nExprt:',Exprt,'\nSqrtAlpha:',SqrtAlpha,'\nInvolSD:',InvolSD )

Delta: 1.0 
BetaStar: 7.701029995663982 
SqrtDelta: 1.0 
RhoC: 0.9993873123068954 
Expat: 0.7788007830714049 
Exprt: 0.9748223789657411 
SqrtAlpha: 0.7071067811865476 
InvolSD: 1.8629008511819873


In [3]:
PriceMatrix = np.zeros((npaths, nsteps+1 ),dtype = np.float32)
PriceMatrix[:,0] = Inital

VolatilityMatrix = np.zeros((npaths, nsteps+1 ),dtype = np.float32)
VolatilityMatrix[:,0] = Sigmat

In [4]:
print(PriceMatrix[:,0:2])
print(PriceMatrix.shape)
print(type(PriceMatrix))

[[15.  0.]
 [15.  0.]
 [15.  0.]
 ...
 [15.  0.]
 [15.  0.]
 [15.  0.]]
(2000, 253)
<class 'numpy.ndarray'>


In [5]:
print(VolatilityMatrix[:,0:2])
print(VolatilityMatrix.shape)
print(type(VolatilityMatrix))

[[0.35 0.  ]
 [0.35 0.  ]
 [0.35 0.  ]
 ...
 [0.35 0.  ]
 [0.35 0.  ]
 [0.35 0.  ]]
(2000, 253)
<class 'numpy.ndarray'>


In [6]:
#Estimating the Volatility and the Sample Paths 
for i in range(1,nsteps+1):
    np.random.seed()
    
    #Estimating the Volatility
    VolMean = BetaStar + Expat*(VolatilityMatrix[:,i-1]-BetaStar)
    Z1 = np.random.standard_normal(npaths)
    VolatilityMatrix[:,i] = VolMean + InvolSD*Z1
    Sigma = np.exp(VolatilityMatrix[:,i])
    
    #Generating the asset path 
    Z2 = np.random.standard_normal(npaths)
    expterm = np.exp(((r - (Sigma*Sigma/2))*Delta) + Sigma*SqrtDelta*(RhoC*Z1 + Rho*Z2))
    PriceMatrix[:,i] = PriceMatrix[:,i - 1] * expterm
    
PriceMatrix = PriceMatrix/Strike

In [7]:
#SMC/LSM, Particle Filter Loop 
MeanMatrix = np.zeros((npaths, nsteps)) 
SdMatrix  = np.zeros((npaths, nsteps))
print(MeanMatrix.shape, SdMatrix.shape)

for i in range(npaths):
    #Return Vector Used to compute weights
    rvec = np.diff(np.log(PriceMatrix[i,:] + np.ones_like(PriceMatrix[i,:])))
    #print(rvec.shape)
    #From Eq * in the Paper, Eq 8 it is the Variance, here it is stdv
    iFiltermean = BetaStar 
    iFilterSD = Gamma/SqrtAlpha
    FiletrSD = Gamma*np.sqrt((1 - np.exp(-2*Alpha*Delta))/(2*Alpha)) 
    
    #Impletenting Equation 8 form the paper also the initialization step in Algorithm1
    OldFilter = np.random.normal(iFiltermean, iFilterSD, nparticle)
    
    for j in range(nsteps):
        FilterMean = BetaStar + Expat*(OldFilter - BetaStar)
        #Step1,Forward Simulation of Algorithm1
        FilterDraw = np.random.normal(0, 1,nparticle)
        NewFilter  = FilterMean + FiletrSD*FilterDraw
        
        Sigma = np.exp(NewFilter)
        
        #Step2, Weighting in Algorithm 1
        WeightMean = (r-Sigma*Sigma/2)*Delta + Sigma*Rho*np.sqrt(Delta)*FilterDraw
        WeightSD = Sigma*SqrtDelta*RhoC
        #print(WeightMean.shape, WeightSD.shape)
        WtVec = scipy.stats.norm.pdf(rvec[j], WeightMean, WeightSD)
        WtVec = np.array(WtVec)
        WtVec /= WtVec.sum()
        #print(np.sum(WtVec))
        #WtVec = scipy.stats.norm.pdf(rvec[j],0,1)
        
        #Step3, Re-Sampling
        Boot_SMC = np.random.choice(NewFilter, size = nparticle, replace=True, p = WtVec)

        PostVec = np.exp(Boot_SMC)
        MeanMatrix[i,j] = np.mean(PostVec)
        SdMatrix[i,j] = np.std(PostVec)

        OldFilter = Boot_SMC
    if (i%100 == 0):
        print((i/npaths)*100,'%Complete')
        

(2000, 252) (2000, 252)
0.0 %Complete
5.0 %Complete
10.0 %Complete
15.0 %Complete
20.0 %Complete
25.0 %Complete
30.0 %Complete
35.0 %Complete
40.0 %Complete
45.0 %Complete
50.0 %Complete
55.00000000000001 %Complete
60.0 %Complete
65.0 %Complete
70.0 %Complete
75.0 %Complete
80.0 %Complete
85.0 %Complete
90.0 %Complete
95.0 %Complete


In [8]:
MeanMatrix

array([[ 2.05215297,  0.64381151,  0.38303503, ...,  0.22745087,
         0.1427122 ,  0.16875276],
       [ 1.9232419 ,  0.41131297,  0.10174538, ...,  0.17069655,
         0.14513755,  0.15724046],
       [ 4.57883435,  1.67261546,  0.67005806, ...,  0.24238124,
         0.1650297 ,  0.19958507],
       ...,
       [ 0.91743683,  0.42521403,  0.24498398, ...,  0.17606926,
         0.14208151,  0.21080482],
       [12.01249386,  0.68152286,  0.2780176 , ...,  0.17260994,
         0.14330847,  0.21107836],
       [ 2.11781281,  0.34540409,  0.23149799, ...,  0.17967781,
         0.23092036,  0.22562222]])

In [9]:
SdMatrix

array([[5.34047291e-01, 7.98383113e-01, 4.39247938e-01, ...,
        3.57429860e-01, 1.87084763e-01, 2.98837155e-01],
       [2.22044605e-16, 5.77160471e-01, 2.03838541e-01, ...,
        3.33740451e-01, 2.41313467e-01, 2.71525008e-01],
       [4.42172567e-01, 8.99034471e-01, 4.76461930e-01, ...,
        3.77669489e-01, 2.56722327e-01, 3.89667367e-01],
       ...,
       [2.22044605e-16, 4.80549837e-01, 3.31787330e-01, ...,
        3.21651734e-01, 2.52342180e-01, 3.57225881e-01],
       [3.55271368e-15, 8.73014007e-01, 3.63140117e-01, ...,
        3.10567942e-01, 1.89192920e-01, 2.72255648e-01],
       [1.02341760e+00, 4.15262727e-01, 3.04528298e-01, ...,
        3.24810130e-01, 3.13533543e-01, 4.56460131e-01]])

In [10]:
#LSM Algorithm for SCM case 
Scale_Strike = 1
PriceMatrix = PriceMatrix[:,1:nsteps+1]
yVec = np.maximum(np.zeros((npaths)), Scale_Strike - PriceMatrix[:,nsteps-1])

strike = 17
for i in range(nsteps-2, 0,-1):
    #Discounted CashFlow 
    yVec = Exprt*yVec
    
    XVec_S = PriceMatrix[:,i]
    XVec_Mu = MeanMatrix[:,i]
    XVec_Sd = SdMatrix[:,i]
    
    # Single terms
    S0 = np.exp(-XVec_S/2)
    S1 = S0*(1 - XVec_S)
    Mu0 = np.exp(-XVec_Mu/2)
    Mu1 = Mu0*(1 - XVec_Mu)
    SD0 = np.exp(-XVec_Sd/2)
    SD1 = SD0*(1 - XVec_Sd)
    
    # Cross_terms
    Cross_1 = S0*Mu0
    Cross_2 = S0*SD0
    Cross_3 = S1*Mu1
    Cross_4 = S1*SD1      
    Cross_5 = Mu0*SD0
    Cross_6 = Mu1*SD1
    
    # Construct the Xmat array.      
    Xmat=np.c_[S0, S1, Mu0, Mu1, SD0, SD1]
    Xmat=np.c_[Xmat, Cross_1, Cross_2, Cross_3, Cross_4, Cross_5, Cross_6]
    
    # Least-squares step
    regression = np.polyfit(yVec, Xmat, 12)
    Hold_Value = np.polyval(regression, Xmat)
    Hold_Value =  np.maximum(np.zeros((npaths)), Hold_Value[:,0])  
    
    Exercise_Value = Scale_Strike - XVec_S
    Exercise_Value = np.maximum(np.zeros((npaths)), Exercise_Value)
    
    CheckVec = Exercise_Value - Hold_Value # Hold value
    
    Evec =[]
    Hvec =[]
    for i in range(int(len(CheckVec))):
        if (CheckVec[i]>=0):
            Evec.append(1)
            Hvec.append(0)
        else:
            Evec.append(0)
            Hvec.append(1)
    Evec= np.array(Evec)
    Hvec = np.array(Hvec)   
    
    for i in range(int(len(Evec))):
        if (Evec[i]==1):
            yVec[i] = Exercise_Value[i]
    yVec = Exprt*Strike*yVec
    
Price_SMCLSM = np.mean(yVec)
SE_SMSLSM    = np.std(yVec) 
print(Price_SMCLSM, SE_SMSLSM);

C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: Ran

C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: Ran

C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: Ran

C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RankWarning: Polyfit may be poorly conditioned
C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: Ran

16.19397055974824 1.6326992704350227


In [11]:
print(Price_SMCLSM, SE_SMSLSM)

16.19397055974824 1.6326992704350227


In [14]:
def CI_95(data):
    a = np.array(data)
    n = len(a)
    m = np.mean(a)
    var = ((np.std(a))**2)*(n/(n-1))
    hw = 1.96*np.sqrt(var/n)
    print('Price of the Option is:',m)
    print('\nThe 95% CI is:',[m-hw,m+hw],'\n')
    return #m, [m-hw,m+hw]

In [15]:
CI_95(yVec)

Price of the Option is: 16.19397055974824

The 95% CI is: [16.12239646349899, 16.26554465599749] 

